# 更多的控制流

如果需要在for循环的过程中进行修改，在迭代过程中做复制。比如通过slice操作：

In [2]:
words = ['cat', 'window', 'defense']
for w in words[:]:
    if len(w) > 3:
        words.insert(0, w)

In [3]:
words

['defense', 'window', 'cat', 'window', 'defense']

默认的参数是在函数定义的时候被验证：

In [10]:
i = 5
def f(arg=i):
    print(arg)
    
i = 6
f()

5


但这只是相对immutable类型来说，对于mutable类型，如list等，<font color='red'>是会不断重新设置默认参数的。</font>

函数内都绑定了固定的默认参数变量，当调用该函数时，没有提供该变量值，则选择绑定的默认变量，而如果默认变量是mutable类型，则它的值会一直变化。一定要注意操作！这个默认变量的属性是定义时候确定的（mutable或者imutable）

In [36]:
def f(a, L=[]):
    L.append(a)
    print(L)

f(1)
f(2)
f(3)

L = []
f(1)
f(2)
f(3)

f(1,[])
f(1)

[1]
[1, 2]
[1, 2, 3]
[1, 2, 3, 1]
[1, 2, 3, 1, 2]
[1, 2, 3, 1, 2, 3]
[1]
[1, 2, 3, 1, 2, 3, 1]


In [37]:
i = [5]
def f(arg=i):
    arg.append(1)
    arg = arg * 2
    print(arg)
    
f()
f()
f()

[5, 1, 5, 1]
[5, 1, 1, 5, 1, 1]
[5, 1, 1, 1, 5, 1, 1, 1]


怎样避免这种情况的产生？！如果默认参数需要改变，则先判断是否为默认值，如果不是，则置为默认值。

In [23]:
def f(a, L=[1]):
    if L != [1]:
        L = [1]
    L.append(a)
    print(L)
    
f(1)
f(2)
f(3)

def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    print(L)
    
f(1)
f(2)
f(3)

[1, 1]
[1, 2]
[1, 3]
[1]
[2]
[3]


但这种代码比较难维护，逻辑混乱，最好常量直接在内部定义，如

In [22]:
def f(a, L=None):
    x = [1]
    if L is None:
        L = x
    L.append(a)
    print(L)
    
f(1)
f(2)
f(3)

[1, 1]
[1, 2]
[1, 3]


### 总结一下

如果默认参数采用imutable类型数据初始化，则后续运算过程中相当于复制一份进行运算，并不影响默认参数的初始化数据。但是如果是mutable类型，则在函数内部调用的时候相当于是引用，共享数据，修改该参数同样会修改绑定的数据。

但比如list数据类型的乘法：
```
a = [1,1,1]
id(a)
a = a * 2 # a = [1,1,1,1,1,1]
id(a)
```
发现a已经不是原来的a了！！！因此这种操作不会改变绑定的数据。
但append操作会改变！

<font color='red'>一般的，单纯的赋值是共享数据，但赋值运算符右侧有其他操作则符则是原数据的拷贝。</font>

关键字参数，```*arguments, **keywords```， 传参的时候需要解析，list和字典分别采用```*， **```

### 4.7.5 lambda表达式

一些小匿名函数可以通过关键字lambda创建。lambda函数可以用在需要函数对象的地方，被语法限制为只有一条表达式。

In [1]:
def make_incrementor(n):
    return lambda x: x + n

f = make_incrementor(2)
f(0)

2

In [3]:
pairs = [(1, 'one'), (2, 'two'), (3, 'three')]
pairs.sort(key=lambda pair: pair[1])
pairs

[(1, 'one'), (3, 'three'), (2, 'two')]

### 4.7.6 文档字符串

接下来是一些惯例，包括文档字符串的内容和格式。

第一行应该是简短，精确描述对象的目的。简单来说，不应该不清晰的陈述对象名字和类型，因为这些对于其他方法是可以获得的。大写开头，句号结束。

如果有多行，第二行应该空行，将摘要和其他描述分开。接下来的行应该分为几段分别描述调用的规则和结果。

python解释器不要求多行缩进，因此如果需要的话工具应该处理缩进。

In [7]:
def my_function():
    '''没有完成什么，只是写个文档
    
    不，真的，什么都没做。
    
不，真的，什么都没做。
    '''
    pass

print(my_function.__doc__)

没有完成什么，只是写个文档
    
    不，真的，什么都没做。
    
不，真的，什么都没做。
    


### 函数注解

函数注解完全是一个用户自定义的可选项。注解保存在__annotations__属性里面，对函数的其他部分没有任何影响。参数注解通过参数名后的冒号定义，后边可接其默认值。返回类型可以通过->定义，后边接返回类型。

In [8]:
def f(ham: str, eggs: str = 'egg') -> str:
    print("Annotations:", f.__annotations__)
    print("Arguments:", ham, eggs)
    return ham + ' and ' +eggs

f('spam')

Annotations: {'ham': <class 'str'>, 'eggs': <class 'str'>, 'return': <class 'str'>}
Arguments: spam egg


'spam and egg'

### 间奏曲： 编码风格

现在你可以开始写更长，更复杂的python程序了，是时候谈谈编码风格。大部分的语言都能有多种风格类型，有一些可读性较好。使别人能够更容易读懂你的代码，接受一种 漂亮的编码风格。

对于python而言，PEP8 已经把风格融合进来了。它支持可读性很高又很好看的编码风格，每一个python开发者应该阅读。这里有一些重要的点：
- 使用4空格缩进，不要制表符
- 一行不超过79个字符
- 使用空行分隔函数和类，以及大代码块之间
- 可能的话，注释单独写一行
- 使用文档字符串
- 操作符和逗号周围添加空格
- 类采用驼峰命名法，函数名使用小写和下划线
- 不要使用花哨的编码方式，utf-8挺好
- 同样的，不要使用非ASCII字符